In [2]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_quant4bit"
model = VGG16_quant()
model.features[24] = QuantConv2d(256, 8, kernel_size=3, stride=1, padding=1, bias=False)
model.features[25] = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
model.features[27] = QuantConv2d(8, 8, kernel_size=3, stride=1, padding=1, bias=False)
model.features[28] = nn.Sequential()
model.features[30] = QuantConv2d(8, 512, kernel_size=3, stride=1, padding=1, bias=False)
#model.features[24] = QuantConv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=False)
#model.features[26] = QuantConv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=False)
#model.features[28] = QuantConv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=False)
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [60, 80]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [3]:
# This cell won't be given, but students will complete the training

lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][0/391]	Time 0.409 (0.409)	Data 0.242 (0.242)	Loss 2.3846 (2.3846)	Prec 10.156% (10.156%)
Epoch: [0][100/391]	Time 0.055 (0.058)	Data 0.002 (0.004)	Loss 2.3539 (3.0093)	Prec 14.062% (11.889%)
Epoch: [0][200/391]	Time 0.054 (0.056)	Data 0.001 (0.003)	Loss 2.1668 (2.5918)	Prec 17.969% (14.385%)
Epoch: [0][300/391]	Time 0.054 (0.056)	Data 0.001 (0.002)	Loss 2.0692 (2.4084)	Prec 22.656% (16.323%)
Validation starts
Test: [0/79]	Time 0.241 (0.241)	Loss 2.0212 (2.0212)	Prec 24.219% (24.219%)
 * Prec 22.950% 
best acc: 22.950000
Epoch: [1][0/391]	Time 0.264 (0.264)	Data 0.212 (0.212)	Loss 1.7874 (1.7874)	Prec 28.906% (28.906%)
Epoch: [1][100/391]	Time 0.058 (0.059)	Data 0.002 (0.004)	Loss 1.9209 (1.9188)	Prec 21.094% (23.987%)
Epoch: [1][200/391]	Time 0.057 (0.057)	Data 0.002 (0.003)	Loss 1.9143 (1.9014)	Prec 26.562% (24.510%)
Epoch: [1][300/391]	Time 0.059 (0.057)	Data 0.002 (0.002)	Loss 1.8668 (1.8936)	Prec 27.344% (24.912%)
Validation starts
Test: [0/79]	Time 0.199 (0.199)	Loss 1.

Epoch: [15][200/391]	Time 0.055 (0.056)	Data 0.001 (0.003)	Loss 0.6446 (0.6516)	Prec 82.812% (77.713%)
Epoch: [15][300/391]	Time 0.057 (0.056)	Data 0.001 (0.002)	Loss 0.6164 (0.6526)	Prec 76.562% (77.634%)
Validation starts
Test: [0/79]	Time 0.213 (0.213)	Loss 0.6141 (0.6141)	Prec 79.688% (79.688%)
 * Prec 76.260% 
best acc: 76.260000
Epoch: [16][0/391]	Time 0.288 (0.288)	Data 0.237 (0.237)	Loss 0.5688 (0.5688)	Prec 82.031% (82.031%)
Epoch: [16][100/391]	Time 0.057 (0.057)	Data 0.002 (0.004)	Loss 0.6380 (0.6152)	Prec 79.688% (79.138%)
Epoch: [16][200/391]	Time 0.055 (0.056)	Data 0.001 (0.003)	Loss 0.7263 (0.6175)	Prec 73.438% (78.910%)
Epoch: [16][300/391]	Time 0.055 (0.056)	Data 0.002 (0.002)	Loss 0.7878 (0.6209)	Prec 75.000% (78.880%)
Validation starts
Test: [0/79]	Time 0.224 (0.224)	Loss 0.6503 (0.6503)	Prec 79.688% (79.688%)
 * Prec 76.210% 
best acc: 76.260000
Epoch: [17][0/391]	Time 0.294 (0.294)	Data 0.247 (0.247)	Loss 0.5400 (0.5400)	Prec 80.469% (80.469%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.085 (0.084)	Data 0.002 (0.002)	Loss 0.3262 (0.3766)	Prec 86.719% (87.256%)
Validation starts
Test: [0/79]	Time 0.231 (0.231)	Loss 0.4799 (0.4799)	Prec 82.031% (82.031%)
 * Prec 80.690% 
best acc: 81.470000
Epoch: [31][0/391]	Time 0.298 (0.298)	Data 0.245 (0.245)	Loss 0.2130 (0.2130)	Prec 93.750% (93.750%)
Epoch: [31][100/391]	Time 0.085 (0.087)	Data 0.002 (0.004)	Loss 0.4575 (0.3508)	Prec 82.812% (88.034%)
Epoch: [31][200/391]	Time 0.088 (0.085)	Data 0.001 (0.003)	Loss 0.2649 (0.3574)	Prec 92.188% (87.885%)
Epoch: [31][300/391]	Time 0.086 (0.085)	Data 0.002 (0.002)	Loss 0.2878 (0.3624)	Prec 89.062% (87.573%)
Validation starts
Test: [0/79]	Time 0.210 (0.210)	Loss 0.4317 (0.4317)	Prec 82.812% (82.812%)
 * Prec 82.780% 
best acc: 82.780000
Epoch: [32][0/391]	Time 0.303 (0.303)	Data 0.237 (0.237)	Loss 0.3235 (0.3235)	Prec 92.188% (92.188%)
Epoch: [32][100/391]	Time 0.088 (0.087)	Data 0.002 (0.004)	Loss 0.2945 (0.3494)	Prec 89.062% (88.475%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.187 (0.187)	Loss 0.3883 (0.3883)	Prec 87.500% (87.500%)
 * Prec 85.640% 
best acc: 85.640000
Epoch: [46][0/391]	Time 0.245 (0.245)	Data 0.184 (0.184)	Loss 0.2375 (0.2375)	Prec 92.969% (92.969%)
Epoch: [46][100/391]	Time 0.082 (0.089)	Data 0.002 (0.004)	Loss 0.2633 (0.2214)	Prec 89.062% (92.396%)
Epoch: [46][200/391]	Time 0.080 (0.087)	Data 0.002 (0.003)	Loss 0.2156 (0.2318)	Prec 92.188% (91.970%)
Epoch: [46][300/391]	Time 0.093 (0.087)	Data 0.002 (0.002)	Loss 0.2348 (0.2356)	Prec 92.188% (91.964%)
Validation starts
Test: [0/79]	Time 0.221 (0.221)	Loss 0.4352 (0.4352)	Prec 85.938% (85.938%)
 * Prec 84.150% 
best acc: 85.640000
Epoch: [47][0/391]	Time 0.255 (0.255)	Data 0.200 (0.200)	Loss 0.1687 (0.1687)	Prec 94.531% (94.531%)
Epoch: [47][100/391]	Time 0.086 (0.085)	Data 0.002 (0.004)	Loss 0.2659 (0.2223)	Prec 88.281% (92.304%)
Epoch: [47][200/391]	Time 0.083 (0.085)	Data 0.002 (0.003)	Loss 0.1895 (0.2186)	Prec 93.750% (92.355%)
Epoch: [47][300/391]	

 * Prec 89.740% 
best acc: 89.740000
Epoch: [61][0/391]	Time 0.382 (0.382)	Data 0.326 (0.326)	Loss 0.1032 (0.1032)	Prec 96.875% (96.875%)
Epoch: [61][100/391]	Time 0.087 (0.089)	Data 0.002 (0.005)	Loss 0.1212 (0.0823)	Prec 92.969% (97.200%)
Epoch: [61][200/391]	Time 0.083 (0.087)	Data 0.002 (0.003)	Loss 0.0938 (0.0836)	Prec 96.094% (97.256%)
Epoch: [61][300/391]	Time 0.084 (0.086)	Data 0.002 (0.003)	Loss 0.0575 (0.0821)	Prec 97.656% (97.282%)
Validation starts
Test: [0/79]	Time 0.192 (0.192)	Loss 0.1908 (0.1908)	Prec 94.531% (94.531%)
 * Prec 89.920% 
best acc: 89.920000
Epoch: [62][0/391]	Time 0.311 (0.311)	Data 0.252 (0.252)	Loss 0.0636 (0.0636)	Prec 97.656% (97.656%)
Epoch: [62][100/391]	Time 0.081 (0.087)	Data 0.001 (0.004)	Loss 0.0622 (0.0701)	Prec 97.656% (97.726%)
Epoch: [62][200/391]	Time 0.083 (0.086)	Data 0.002 (0.003)	Loss 0.0758 (0.0756)	Prec 96.875% (97.520%)
Epoch: [62][300/391]	Time 0.083 (0.086)	Data 0.001 (0.002)	Loss 0.0882 (0.0751)	Prec 97.656% (97.498%)
Validation s

Epoch: [76][100/391]	Time 0.082 (0.087)	Data 0.002 (0.004)	Loss 0.0505 (0.0362)	Prec 97.656% (98.708%)
Epoch: [76][200/391]	Time 0.088 (0.085)	Data 0.001 (0.003)	Loss 0.0440 (0.0388)	Prec 98.438% (98.644%)
Epoch: [76][300/391]	Time 0.085 (0.085)	Data 0.001 (0.002)	Loss 0.0112 (0.0387)	Prec 100.000% (98.624%)
Validation starts
Test: [0/79]	Time 0.229 (0.229)	Loss 0.2904 (0.2904)	Prec 91.406% (91.406%)
 * Prec 90.140% 
best acc: 90.320000
Epoch: [77][0/391]	Time 0.270 (0.270)	Data 0.206 (0.206)	Loss 0.0074 (0.0074)	Prec 100.000% (100.000%)
Epoch: [77][100/391]	Time 0.086 (0.087)	Data 0.002 (0.004)	Loss 0.0686 (0.0370)	Prec 96.875% (98.801%)
Epoch: [77][200/391]	Time 0.085 (0.086)	Data 0.002 (0.003)	Loss 0.0173 (0.0374)	Prec 99.219% (98.795%)
Epoch: [77][300/391]	Time 0.084 (0.085)	Data 0.002 (0.002)	Loss 0.0822 (0.0363)	Prec 98.438% (98.785%)
Validation starts
Test: [0/79]	Time 0.211 (0.211)	Loss 0.2566 (0.2566)	Prec 94.531% (94.531%)
 * Prec 90.270% 
best acc: 90.320000
Epoch: [78][0/39

Epoch: [91][200/391]	Time 0.091 (0.086)	Data 0.002 (0.003)	Loss 0.0268 (0.0284)	Prec 99.219% (99.024%)
Epoch: [91][300/391]	Time 0.085 (0.085)	Data 0.002 (0.003)	Loss 0.0310 (0.0280)	Prec 98.438% (99.068%)
Validation starts
Test: [0/79]	Time 0.248 (0.248)	Loss 0.3416 (0.3416)	Prec 92.969% (92.969%)
 * Prec 90.410% 
best acc: 90.630000
Epoch: [92][0/391]	Time 0.281 (0.281)	Data 0.223 (0.223)	Loss 0.0077 (0.0077)	Prec 100.000% (100.000%)
Epoch: [92][100/391]	Time 0.088 (0.087)	Data 0.002 (0.004)	Loss 0.0732 (0.0275)	Prec 96.875% (99.002%)
Epoch: [92][200/391]	Time 0.097 (0.087)	Data 0.002 (0.003)	Loss 0.0237 (0.0268)	Prec 99.219% (99.052%)
Epoch: [92][300/391]	Time 0.086 (0.087)	Data 0.002 (0.003)	Loss 0.0730 (0.0265)	Prec 98.438% (99.073%)
Validation starts
Test: [0/79]	Time 0.217 (0.217)	Loss 0.2975 (0.2975)	Prec 91.406% (91.406%)
 * Prec 90.350% 
best acc: 90.630000
Epoch: [93][0/391]	Time 0.273 (0.273)	Data 0.214 (0.214)	Loss 0.0247 (0.0247)	Prec 99.219% (99.219%)
Epoch: [93][100/391

In [10]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
device = torch.device("cuda" if use_gpu else "cpu") 
counter =0
for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print("prehooked")
        counter += 1
        print(layer, counter)
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)


prehooked
QuantConv2d(
  3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 1
prehooked
QuantConv2d(
  64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 2
prehooked
QuantConv2d(
  64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 3
prehooked
QuantConv2d(
  128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 4
prehooked
QuantConv2d(
  128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 5
prehooked
QuantConv2d(
  256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 6
prehooked
QuantConv2d(
  256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 7
prehooked
QuantConv2d(
  2

In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [3]:
PATH = "result/VGG16_quant4bit/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)



Test set: Accuracy: 9062/10000 (91%)



In [ ]:
print(len(save_output.outputs))

In [ ]:
len(save_output.outputs[0][0])

In [66]:
w_bit = 4
weight_q = model.features[27].weight_q # quantized value is stored during the training
w_alpha = model.features[27].weight_quant.wgt_alpha
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[ 7.0000, -7.0000,  7.0000],
          [-2.0000,  7.0000, -7.0000],
          [-7.0000,  7.0000, -7.0000]],

         [[ 7.0000, -7.0000, -7.0000],
          [ 7.0000,  7.0000,  7.0000],
          [ 7.0000,  7.0000,  7.0000]],

         [[ 7.0000,  7.0000,  7.0000],
          [ 7.0000,  7.0000,  7.0000],
          [-7.0000, -7.0000, -7.0000]],

         [[ 7.0000, -7.0000,  7.0000],
          [-7.0000, -7.0000, -7.0000],
          [ 7.0000, -7.0000,  7.0000]],

         [[-7.0000, -7.0000,  7.0000],
          [ 7.0000,  7.0000, -7.0000],
          [ 7.0000, -7.0000, -7.0000]],

         [[ 7.0000,  7.0000,  7.0000],
          [ 7.0000,  7.0000, -7.0000],
          [ 7.0000, -7.0000, -7.0000]],

         [[-7.0000, -7.0000, -7.0000],
          [ 7.0000,  7.0000, -7.0000],
          [-7.0000, -7.0000,  7.0000]],

         [[-7.0000,  7.0000,  7.0000],
          [-7.0000,  7.0000,  7.0000],
          [ 7.0000,  7.0000,  7.0000]]],


        [[[ 7.0000,  7.0000,  7.0000],
       

In [67]:
x_bit = 4
x = save_output.outputs[8][0]  # input of the 8th conv layer
x_alpha  = model.features[27].act_alpha
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 5.0000,  2.0000,  2.0000,  1.0000],
          [ 5.0000,  0.0000,  2.0000,  0.0000],
          [ 4.0000,  1.0000,  4.0000,  2.0000],
          [ 4.0000,  2.0000,  4.0000,  3.0000]],

         [[ 2.0000,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  0.0000,  0.0000,  1.0000],
          [ 2.0000,  3.0000,  4.0000,  3.0000],
          [ 2.0000,  3.0000,  4.0000,  3.0000]],

         ...,

         [[ 1.0000,  0.0000,  0.0000,  0.0000],
          [ 1.0000,  1.0000,  0.0000,  2.0000],
          [ 2.0000,  1.0000,  0.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  1.0000]],

         [[ 1.0000,  2.0000,  2.0000,  1.0000],
          [ 2.0000,  3.0000,  4.0000,  3.0000],
          [ 1.0000,  2.0000,  3.0000,  2.0000],
          [ 2.0000,  2.0000,  1.0000,  0.0000]],

         [[ 0.0

In [92]:
x_int.size()

torch.Size([128, 8, 4, 4])

In [93]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
relu = nn.ReLU()
output_int = conv_int(x_int)
output_recovered = output_int*w_delta*x_delta
output_recovered = relu(output_recovered)

In [94]:
difference = abs(save_output.outputs[9][0] - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(2.2645e-07, device='cuda:0', grad_fn=<MeanBackward0>)


In [116]:
x_pad = torch.zeros(128, 8, 6, 6).cuda()

In [117]:
x_pad[ : ,  :, 1:5, 1:5] = x_int.cuda()

In [118]:
X = x_pad[0]
X = torch.reshape(X, (X.size(0), -1))

In [119]:
X.size()

torch.Size([8, 36])

In [120]:
tile_id = 0 
nij = 200 # just a random number
#X = a_tile[tile_id,:,nij:nij+64]  # [tile_num, array row num, time_steps]

bit_precision = 4
file = open('activation.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(int(X[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file 

In [121]:
X[:,7]

tensor([0., 5., 2., 2., 1., 1., 1., 0.], device='cuda:0',
       grad_fn=<SelectBackward>)

In [122]:
weight_int.size()

torch.Size([8, 8, 3, 3])

In [123]:
weight_int.size() # 8, 8 , 3, 3
W = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))
W.size() # 8, 8, 9

torch.Size([8, 8, 9])

In [124]:
bit_precision = 4
file = open('weight.txt', 'w') #write to file
file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
file.write('#................#\n')
for kij in range(9):
    for i in range(W.size(0)):  
        for j in range(W.size(1)):
            if (W[i, 7-j, kij].item()<0):
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+2**bit_precision+0.001))
            else:
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+0.001))
            for k in range(bit_precision):
                file.write(W_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close() #close file  



In [125]:
W[0,:,0]

tensor([ 7.0000,  7.0000,  7.0000,  7.0000, -7.0000,  7.0000, -7.0000, -7.0000],
       device='cuda:0', grad_fn=<SelectBackward>)

In [126]:
p_nijg = range(X.size(1)) ## psum nij group

psum = torch.zeros(8, len(p_nijg), 9).cuda() 

In [127]:
psum.size()

torch.Size([8, 36, 9])

In [128]:
for kij in range(9):  
    for nij in p_nijg:       # time domain, sequentially given input
        m = nn.Linear(8, 8, bias=False)
        m.weight = torch.nn.Parameter(W[:,:,kij])
        psum[:, nij, kij] = m(X[:,nij]).cuda()

In [129]:



bit_precision = 16
file = open('psum.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')
for kij in range(9):
    for i in range(psum.size(1)):
        for j in range(psum.size(0)):
            if (psum[7-j,i,kij].item()<0):
                P_bin = '{0:016b}'.format(int(psum[7-j,i,kij].item()+2**bit_precision+0.001))
            else:
                P_bin = '{0:016b}'.format(int(psum[7-j,i,kij].item()+0.001))
            for k in range(bit_precision):
                file.write(P_bin[k])
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close()

In [130]:
psum[:,8,0]

tensor([ 14.0000, -28.0000,  42.0000,  14.0000, -28.0000, -14.0000, -28.0000,
        -14.0000], device='cuda:0', grad_fn=<SelectBackward>)

In [6]:

address = torch.zeros(16, 9).cuda()

for o_nij in range(16):
    for kij in range(9):
        address[o_nij, kij] = int(o_nij/4)*6 + o_nij%4 + int(kij/3)*6 + kij%3
        #print(address[o_nij, kij])

tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(2., device='cuda:0')
tensor(6., device='cuda:0')
tensor(7., device='cuda:0')
tensor(8., device='cuda:0')
tensor(12., device='cuda:0')
tensor(13., device='cuda:0')
tensor(14., device='cuda:0')
tensor(1., device='cuda:0')
tensor(2., device='cuda:0')
tensor(3., device='cuda:0')
tensor(7., device='cuda:0')
tensor(8., device='cuda:0')
tensor(9., device='cuda:0')
tensor(13., device='cuda:0')
tensor(14., device='cuda:0')
tensor(15., device='cuda:0')
tensor(2., device='cuda:0')
tensor(3., device='cuda:0')
tensor(4., device='cuda:0')
tensor(8., device='cuda:0')
tensor(9., device='cuda:0')
tensor(10., device='cuda:0')
tensor(14., device='cuda:0')
tensor(15., device='cuda:0')
tensor(16., device='cuda:0')
tensor(3., device='cuda:0')
tensor(4., device='cuda:0')
tensor(5., device='cuda:0')
tensor(9., device='cuda:0')
tensor(10., device='cuda:0')
tensor(11., device='cuda:0')
tensor(15., device='cuda:0')
tensor(16., device='cuda:0')
tensor

In [5]:
address.size()

torch.Size([16, 9])

In [8]:

file = open('acc_address.txt', 'w') #write to file
file.write('#1st address#\n')
file.write('#2st address#\n')
file.write('#................#\n')
bit_precision = 11

for i in range(address.size(0)):
    for j in range(address.size(1)):
        a_bin = '{0:011b}'.format(int(address[i, j]))
        for k in range(bit_precision):
            file.write(a_bin[k])
        file.write('\n')
file.close()

In [131]:
output_int.size()

torch.Size([128, 8, 4, 4])

In [132]:
out = output_int[0]
out.size()

torch.Size([8, 4, 4])

In [133]:
out = torch.reshape(out, (out.size(0), -1))

In [134]:
out.size()

torch.Size([8, 16])

In [135]:
bit_precision = 16
file = open('output.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(out.size(1)):
    for j in range(out.size(0)):
        if (out[7-j,i].item()<0):
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+2**bit_precision+0.001))
        else:
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(O_bin[k])
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close()

In [136]:
out[:,4]

tensor([ -56.0000,  -20.0000,   84.0000, -112.0000,   84.0000,  -76.0000,
        -126.0000,  140.0000], device='cuda:0', grad_fn=<SelectBackward>)